# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [23]:
# Write your code below.
%load_ext dotenv
%dotenv 

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [24]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [25]:
import os
from glob import glob

# Write your code below.

PRICE_DATA = os.getenv("PRICE_DATA")

parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive=True)
print(parquet_files)


['../../05_src/data/prices\\ACN\\ACN_2001\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2001\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2002\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2002\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2003\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2003\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2004\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2004\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2005\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2005\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2006\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2006\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2007\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2007\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2008\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2008\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2009\\part.0.parque

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [26]:
# Write your code below.
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

# Add lags for variables Close and Adj_Close
dd_shift = (
    dd_px
    .groupby('ticker', group_keys=False)
    .apply(lambda x: x.assign(
        Close_lag_1 = x['Close'].shift(1), 
        Adj_Close_lag_1 = x['Adj Close'].shift(1)))
)

#Add returns and hi_lo_range
dd_feat = dd_shift.assign(
    Returns = lambda x: x['Close']/x['Close_lag_1'] - 1,
    Hi_lo_range = lambda x: x['High']-x['Low']
)

dd_feat


C:\Users\blamo\AppData\Local\Temp\ipykernel_30328\3396824353.py:6: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_px


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,Returns,Hi_lo_range
npartitions=90,,,,,,,,,,,,,
ACN,datetime64[ns],float64,float64,float64,float64,float64,float64,string,int32,float64,float64,float64,float64
ADP,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [28]:
# Write your code below.
#Convert Dask data frame to pandas data frame
pd_feat = dd_feat.compute()

#Add a moving average of returns for a 10 day window to the data frame
pd_feat['MA_returns_10'] = pd_feat['Close'].rolling(10).mean()

pd_feat


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,Returns,Hi_lo_range,MA_returns_10
ticker,,,,,,,,,,,,,,
ACN,2017-07-05,124.000000,124.779999,123.989998,124.050003,118.719963,1328600.0,ACN.csv,2017,NaN,NaN,NaN,0.790001,NaN
ACN,2017-07-06,123.349998,123.629997,122.029999,122.940002,117.657669,1938100.0,ACN.csv,2017,124.050003,118.719963,-0.008948,1.599998,NaN
ACN,2017-07-07,123.550003,124.750000,123.070000,124.209999,118.873100,1889400.0,ACN.csv,2017,122.940002,117.657669,0.010330,1.680000,NaN
ACN,2017-07-10,124.050003,124.370003,123.589996,124.059998,118.729546,1301500.0,ACN.csv,2017,124.209999,118.873100,-0.001208,0.780006,NaN
ACN,2017-07-11,124.059998,124.099998,123.199997,123.849998,118.528572,1379800.0,ACN.csv,2017,124.059998,118.729546,-0.001693,0.900002,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,2003-12-24,7.650000,7.960000,7.500000,7.740000,7.740000,236300.0,ZIXI.csv,2003,7.620000,7.620000,0.015748,0.460000,7.011
ZIXI,2003-12-26,7.900000,8.130000,7.800000,8.060000,8.060000,144200.0,ZIXI.csv,2003,7.740000,7.740000,0.041344,0.330000,7.142
ZIXI,2003-12-29,8.000000,8.100000,7.810000,7.880000,7.880000,245400.0,ZIXI.csv,2003,8.060000,8.060000,-0.022333,0.290000,7.214


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
Converting to pandas was a faster and simpler way to calculate the moving average return since it fits easily into memory.

+ Would it have been better to do it in Dask? Why?
For larger dataset it would be better to do it Dask as it computes lazily and only executes when .compute() is called.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.